In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
from PIL import Image

def pad_frames_to_multiple_of_12(frames, target_dir):
    """12배수보다 4프레임 이하로 부족하면 마지막 프레임을 복사해서 12배수로 맞춤"""
    num_frames = len(frames)
    remainder = num_frames % 12

    if remainder > 0 and (12 - remainder) <= 4:  # (12 - remainder)가 4 이하일 때만 패딩
        last_frame = frames[-1]
        for _ in range(12 - remainder):
            new_frame_path = os.path.join(target_dir, f"frame_{num_frames}.jpg")
            shutil.copy(last_frame, new_frame_path)
            num_frames += 1

def process_video_folders(base_dir, output_dir):
    """영상 폴더 내 프레임을 12배수 단위로 분할하고, 부족한 경우 보정"""

    drop_count = 0

    # 모든 영상 폴더를 순회
    for video_folder in os.listdir(base_dir):
        video_path = os.path.join(base_dir, video_folder)

        if not os.path.isdir(video_path):
            continue  # 폴더가 아닐 경우 스킵

        frames = sorted([os.path.join(video_path, f) for f in os.listdir(video_path) if f.endswith('.jpg')])

        if len(frames) < 8:  # 0~7개 프레임만 있는 폴더는 제외
            print(f"Skipping {video_folder} (only {len(frames)} frames)")
            continue

        num_frames = len(frames)
        num_subfolders = num_frames // 12  # 12배수로 자를 수 있는 그룹 개수
        remainder = num_frames % 12  # 나머지 프레임 수

        # 새로운 폴더 생성 및 프레임 복사
        for i in range(num_subfolders):
            subfolder_path = os.path.join(output_dir, f"{video_folder}_part{i+1}")
            os.makedirs(subfolder_path, exist_ok=True)

            start_idx = i * 12
            end_idx = start_idx + 12
            for frame_path in frames[start_idx:end_idx]:
                shutil.copy(frame_path, subfolder_path)

        # 나머지 프레임 처리
        if remainder > 0:
            last_part_path = os.path.join(output_dir, f"{video_folder}_part{num_subfolders+1}")
            os.makedirs(last_part_path, exist_ok=True)

            last_frames = frames[-remainder:]  # 남은 프레임
            if (12 - remainder) <= 4:  # 4개 이하 부족하면 마지막 프레임 복사해서 12배수로 맞추기
                pad_frames_to_multiple_of_12(last_frames, last_part_path)  # 마지막 프레임 복사해서 채우기
            elif (12 - remainder) > 4:
                print(f"Dropping {remainder} frames from {video_folder}")  # 4프레임 초과하면 삭제
                drop_count += 1

        print(f"Processed {video_folder}: {num_subfolders} folders created.")

    print("----Complete----")
    print(f"total drop count = {drop_count}")

In [ ]:
# 실행 경로 설정
base_directory = "/content/drive/MyDrive/DMS/data/DMD/dmd_frames"  # 원본 프레임 폴더들이 있는 경로
output_directory = "/content/drive/MyDrive/DMS/data/DMD/dmd_frames_12x"  # 정리된 프레임이 저장될 경로

process_video_folders(os.path.join(base_directory, 'Text'), os.path.join(output_directory, 'Text'))

Skipping gE-30(File20)_s2_face_2019-03-15-10;43;40_30_1.avi (only 3 frames)
Skipping gE-28(File18)_s2_face_2019-03-15-10;12;30_28_0.avi (only 4 frames)
Dropping 4 frames from gE-28(File18)_s2_face_2019-03-15-10;12;30_28_1.avi
Processed gE-28(File18)_s2_face_2019-03-15-10;12;30_28_1.avi: 9 folders created.
Processed gE-27(File17)_s2_face_2019-03-07-13;05;11_27_1.avi: 9 folders created.
Dropping 6 frames from gE-28(File18)_s2_face_2019-03-15-10;12;30_28_2.avi
Processed gE-28(File18)_s2_face_2019-03-15-10;12;30_28_2.avi: 11 folders created.
Dropping 3 frames from gZ-37(File31)_s2_face_2019-04-08-15;45;15_37_0.avi
Processed gZ-37(File31)_s2_face_2019-04-08-15;45;15_37_0.avi: 6 folders created.
Processed gZ-37(File31)_s4_face_2019-04-03-15;40;42_37_0.avi: 11 folders created.
Dropping 7 frames from gB-6(File6)_s2_face_2019-03-11-13;46;14_6_4.avi
Processed gB-6(File6)_s2_face_2019-03-11-13;46;14_6_4.avi: 8 folders created.
Dropping 1 frames from gB-9(File9)_s4_face_2019-03-22-09;43;35_9_1.avi